# How magnetic waves are used to construct non-invasive internal anatomic and physiologic images 

Mira Liu 9/09/2022


## Let's start with how the way we hear sounds is similar to how we image with MRI:

When plucking a string, one can hear a note. That note comes from oscillation at a particular frequency. Notice one note has one frequency, while two notes will have two frequencies, and three notes will have three frequencies. This can be shown with the guitar! 

Imagine how useful it would be if instead of seeing just the sum of those sinusoids, instead we could get the frequency, amplitude, and phase of the sinusoids that are making up that note!

This is called a "fourier transform". This means that every single signal can be decomposed into the sum of sinusoids of varying frequencies. In the fourier domain, it is represented by a delta function at the frequency of the wave, with the amplitude and phase saved as the height of the delta function and the complex portion of the signal. 

In [31]:
# Load packages 

import numpy as np
import matplotlib
import matplotlib.pyplot as pl

import random
import pickle
import seaborn as sns
import csv
import pandas as pd
import sys 
import os
import matplotlib.image as mpimg
from mpl_toolkits.mplot3d import Axes3D
import pydicom as dicom

# Load Data
    
with open('data/Fig1_1dWaves.pickle','rb') as handle: 
    wave1,ft_sin1,wave2,ft_sin2,wavesum,ft_sinsum = pickle.load(handle)

with open('data/Fig2_2dWaves.pickle','rb') as handle: 
    img,kspace = pickle.load(handle)
    
with open('data/Fig3_Rotating.pickle','rb') as handle: 
    image_rotating,kspace_rotating = pickle.load(handle)
    
with open('data/Fig4_RotatingIncreasing.pickle','rb') as handle: 
    image_rotateincrease,kspace_rotateincrease = pickle.load(handle)


# Here's an example with a sound wave of 5Hz and a sound wave of 7Hz: 

Starting with one sinusoid, we can see the fourier transform of it in the fourier domain and how it shifts location as the frequency increases! 

Then, we can see how the combination of the two can be shown as the sum of two sinusoids in fourier space!

In [2]:
%matplotlib

Using matplotlib backend: MacOSX


In [6]:
x = np.linspace(0, 1, 1000) #from 0 to 1, sampling rate of 1/1000
ft_freq = np.linspace(-500,500,1000) #kmax = 1/2Deltax, Delta kx = 1/FOVx

# Set up subplots
fig, ((ax1, ax2),(ax3,ax4),(ax5,ax6)) = pl.subplots(3,2)

# plot wave 1 with 5 cycles per second
ax1.plot(x,wave1)
ax1.set_title('5 Hz')
ax2.plot(ft_freq,abs(ft_sin1))
ax2.set_title('FT of 5 Hz')
ax2.set_xlim([-10,10])

# plot wave 2 with 7 cycles per second
ax3.plot(x,wave2,color = 'red')
ax3.set_title('7 Hz')
ax4.plot(ft_freq,abs(ft_sin2),color = 'red')
ax4.set_title('FT of 7 Hz')
ax4.set_xlim([-10,10])

# plot sum of the two
ax5.plot(x,wave1+wave2,color = 'purple')
ax5.set_title('5 Hz + 7 Hz')
ax6.plot(ft_freq,abs(ft_sinsum),color = 'purple')
ax6.set_title('FT of 7 Hz + 5 Hz')
ax6.set_xlim([-10,10])


fig.tight_layout()
pl.show()


# Now how can we see this in an image? 

Let's make those waves two dimensional! They're planar waves now, 2D! 

Of course, this means they will be two frequency space? 

That doesn't make any sense intuitively, does it... so let's take a look!

Here you see that the 2D planar wave of a single frequency is again actually just two delta functions in a 2D plane! 

So what does their location in the 2D space mean? 

In [5]:
pl.close()    
fig, ((ax1, ax2),(ax3,ax4)) = pl.subplots(2,2)

x = np.linspace(0, 1, 100)
ft_freq = np.linspace(-50,50,100)
ax1.imshow(img,cmap = 'gray',extent = [0,1,0,1])
ax1.set_title('2d planar wave')
ax2.plot(x,img[51])
ax2.set_title('Cross section of 2d planar wave')

#so this has wavelength of 30
# pixel size of 3

ax3.imshow(np.abs(kspace),cmap = 'gray',extent = [-50,50,-50,50])
ax3.set_title('Fourier Transform of 2d planar wave')
ax4.plot(ft_freq,np.abs(kspace)[50])
ax4.set_title('Cross section of Fourier Transform')
fig.tight_layout()
pl.show()


#pl.savefig('data/Fig2.png',dpi = 100)

#with open('data/Fig2_2dWaves.pickle','wb') as handle: 
    #pickle.dump([img,kspace],handle)


# What does 2D wave mean?

A 1D wave doesn't have any direction, it only has frequency. (this can be hard to understand, but if you think about it, even for a 1D wave to travel in a direction, that direction has to be a direction in at least 2D space!) 

But a 2D wave has both a frequency AND a direction in which it is going! This means that the direction of the frequency is also encoded in its location in the 2D frequency space.

As we rotate the planar waves to have them propagating in different directions, we can see the delta functions also rotating in k-space!

Below we can see that the location of the delta waves in the 2D space is the direction of the wave! One 2D wave (5Hz) is rotated about the center, and we can see that the delta waves in 2D space rotate with it.

In [16]:
pl.close() 
fig, (ax1, ax2) = pl.subplots(1,2)
for j in range(np.shape(image_rotating)[2]):
    ax1.imshow(image_rotating[:,:,j],cmap = 'gray',extent = [0,1,0,1])
    ax1.set_title('Rotating\nPlanar Wave')
    ax2.imshow(np.abs(kspace_rotating[:,:,j]),cmap = 'gray',extent = [-50,50,-50,50])
    ax2.set_title('Corresponding FT\n"k-space"')

    pl.pause(.5)
    

# What about the 2D frequency though? 

How is the frequency of the wave encoded then? 

Remember the 1D wave? The 5Hz wave was 5 away from the center, while the 7Hz wave was 7 away from the center. In 2D space then, a 2D 5Hz wave will be 5 from the 2D origin (0,0) while aa 7Hz wave will be 7 away from the 2D origin! 

This means that the frequency is from the distance of the delta function from the center, while the direction of the wave is the angle of the delta function from the center! 

Below is an example of a wave that is increasing in frequency while rotating. We can see that the fourier transform is showing the delta waves are moving away from the center and rotating as well. 

In [29]:
pl.close() 
fig, (ax1, ax2) = pl.subplots(1,2)
for j in range(np.shape(image_rotating)[2]):
    ax1.imshow(image_rotateincrease[:,:,j],cmap = 'gray',extent = [0,1,0,1])
    ax1.set_title('Rotating & Increasing Hz\nPlanar Wave')
    ax2.imshow(np.abs(kspace_rotateincrease[:,:,j]),cmap = 'gray',extent = [-50,50,-50,50])
    ax2.set_title('Corresponding FT\n"k-space"')

    pl.pause(.5)
    

# From this MRI can make images! 

We can now see how a 2D wave can rotate and change frequencies, how do we get images from that? We can add 2D waves! 

Just like the first image with the addition of two 1D waves, we can do exactly that with 2D waves. MRIs can use magnetic fields to measure the frequency, direction, and phase of the 2D wavepatterns of tissue. By imaging this we can get the individual contribution of each type of wave, and from this we can get images of biologic anatomy and physiology. 

Here is an example of an MRI brain scan. 


In [39]:
name = 'Coronal' #Sagittal, Axial

fov_img = 256

with open('data/'+name + '_superposition.pickle','rb') as handle1:
    stack_image = pickle.load(handle1)
with open('data/'+ name +'_superposition_masks.pickle','rb') as handle2:
    stack_waves = pickle.load(handle2)
with open('data/'+ name + '_superposition_kspace.pickle','rb') as handle3:
    stack_kspace = pickle.load(handle3)
with open('data/'+ name + '_FullImages.pickle','rb') as handle4:
    img,kspace = pickle.load(handle4)
    
    
fig, (ax1, ax2, ax3,ax4) = pl.subplots(1,4)
ax4.imshow(img,cmap = 'gray')
ax4.set_title('Final Image')
for i in np.arange(0,np.shape(stack_image)[2]):
    ax1.imshow(stack_kspace[:,:,i],vmin = 0,vmax = 8000,extent = [-.5, .5, -.5, .5],cmap = 'gray')
    ax1.set_title('k-space')
    ax2.imshow(stack_waves[:,:,i],cmap = 'gray')
    ax2.set_title('planar\nwaves')
    ax3.imshow(stack_image[:,:,i],cmap = 'gray')
    ax3.set_title('Image\nForming')
    pl.pause(.1)

In [42]:
fig, (ax1, ax2) = pl.subplots(1,2)

ax1.imshow(np.abs(kspace),vmin = 0,vmax = 8000,extent = [-.5, .5, -.5, .5],cmap = 'gray')
ax1.set_title('k-space collected \nwith MRI')
ax2.imshow(img,cmap = 'gray')
ax2.set_title('Image from MRI')
pl.show()
